In [1]:
using Distributed

using Pkg


@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [2]:
using Random
@everywhere import ..MCRun: initialise_histograms!,updatehistogram!,update_max_stepsize!,sampling_step!,save_results,save_states
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 50.
tf = 160.
n_traj = 16

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
ico_13  = [[-0.0000000049,       -0.0000000044,       -0.0000000033],
[-0.0000007312,       -0.0000000014,        0.6554619119],
 [0.1811648930,      -0.5575692094,        0.2931316798],
[-0.4742970242,       -0.3445967289,        0.2931309525],
[-0.4742970303,        0.3445967144,        0.2931309494],
 [0.1811648830,        0.5575692066,        0.2931316748],
 [0.5862626299,        0.0000000022,        0.2931321262],
[-0.1811648928,       -0.5575692153,       -0.2931316813],
[-0.5862626397,       -0.0000000109,       -0.2931321327],
[-0.1811649028,        0.5575692007,       -0.2931316863],
 [0.4742970144,        0.3445967202,       -0.2931309590],
 [0.4742970205,       -0.3445967231,       -0.2931309559],
 [0.0000007214,       -0.0000000073,       -0.6554619185]]

#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu13 = copperconstant*ico_13*1.5
AtoBohr = 1.8897259886


#boundary conditions 
bc_cu13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_cu13, bc_cu13)

#histogram information
n_bin = 100




100

In [3]:
# function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
#     if isdir("pots") == true
#         rm("pots",recursive = true)
#     end
#     potential_directory = mkdir("$(pwd())/pots")
#     potential_vector = []
#     for i = 1:n_threads
#         # n_threads potentials
#         cp(pot.dir,"$(potential_directory)/potential$i")
#         pot_i = AbstractMLPotential("$(potential_directory)/potential$i/",pot.atomtype)
#         push!(potential_vector,pot_i)
#     end
        
#     return potential_vector        
# end

function parallel_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)
    parallel_states = []

     #initialise state and potentials

    n_threads = Threads.nthreads()
    sample_index = Int64(floor(mc_params.eq_cycles / n_threads)) #number of eq cycles per thread

    a = atom_move_frequency(move_strat)
    v = vol_move_frequency(move_strat)
    r = rot_move_frequency(move_strat)# function init_parallel_RuNNer(pot::AbstractMLPotential; n_threads = Threads.nthreads())
    n_steps = a + v + r
    println()

    ebounds = [100. , -100.]

    for i_thread = 1:n_threads
        for i_eq = 1:sample_index
            
            i = i_thread*sample_index + i_eq

            mc_states = mc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)#mc cycle
            
            for i_traj = 1:mc_params.n_traj#check energy bounds
                if mc_states[i_traj].en_tot < ebounds[1]
                    ebounds[1] = mc_states[i_traj].en_tot
                end

                if mc_states[i_traj].en_tot > ebounds[2]
                    ebounds[2] = mc_states[i_traj].en_tot
                end

            end

            if rem(i, mc_params.n_adjust) == 0 #adjust stepsize
                for i_traj = 1:mc_params.n_traj
                    update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
                end 
            end

        end
        
        states_vec = [MCState(mc_states[i_traj].temp,mc_states[i_traj].beta,mc_states[i_traj].config,pot_vector[i_thread];max_displ = mc_states[i_traj].max_displ ) for i_traj in 1:mc_params.n_traj] #initialise a new mc_states vector based on current state

        for i_traj = 1:mc_params.n_traj
            push!(states_vec[i_traj].ham, 0)
            push!(states_vec[i_traj].ham, 0)
        end

        push!(parallel_states,states_vec) #add to vector of parallel states


    end
    
    

    delta_en = initialise_histograms!(mc_params,mc_states,results, full_ham=false,e_bounds=ebounds) #start histogram

    println("equilibration done")
    
    return parallel_states,a,v,r,delta_en,n_threads

end

parallel_equilibration (generic function with 1 method)

In [4]:
import .MCRun.mc_cycle!
function mc_cycle!(mc_states, move_strat, mc_params, pot::ParallelMLPotential, ensemble, n_steps, a, v, r)
    file = RuNNer.writeinit(pot.dir;input_idx=pot.index)
    #this for loop creates n_traj perturbed atoms
    indices = []
    trials = []
    #we require parallelisation here, but will need to avoid a race condition
    for mc_state in mc_states
        #for i_step = 1:n_steps
            ran = rand(1:(a+v+r))
            trial_pos = atom_displacement(mc_state.config.pos[ran], mc_state.max_displ[1], mc_state.config.bc)
            writeconfig(file,mc_state.config,ran,trial_pos, pot.atomtype)
            push!(indices,ran)
            push!(trials,trial_pos)
        #end
    end
    #after which we require energy evaluations of the n_traj new configurations
    close(file)    
    energyvec = getRuNNerenergy(pot.dir,mc_params.n_traj; input_idx=pot.index)    
    #this replaces the atom_move! function
    #parallelisation here is fine
    for i in 1:mc_params.n_traj
        if metropolis_condition(ensemble, (mc_states[i].en_tot - energyvec[i]), mc_states[i].beta ) >=rand()
            mc_states[i].config.pos[indices[i]] = trials[i]
            mc_states[i].en_tot = energyvec[i]
            mc_states[i].count_atom[1] +=1
            mc_states[i].count_atom[2] += 1
        end
    end


    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end


    return mc_states
end

mc_cycle! (generic function with 3 methods)

In [5]:
@everywhere function threadexchange(parallel_states,n_threads,idx)
    if rand() < 0.2  #20% change per trajectory of an attempted exchange
        thrid = rand(1:n_threads,2)
        if thrid[1] == thrid[2] && thrid[2] == n_threads
            thrid[2] = rand(1:n_threads-1)
        elseif thrid[1] == thrid[2] && thrid[2] != n_threads
            thrid[2] +=1
        end #which threads are talking

        exc_acc = exc_acceptance(parallel_states[thrid[1]][idx].beta,parallel_states[thrid[2]][idx].beta,parallel_states[thrid[1]][idx].en_tot,parallel_states[thrid[2]][idx].en_tot) #calc acceptance
        
        if exc_acc > rand() #if exchange is likely
            exc_trajectories!(parallel_states[thrid[1]][idx] ,parallel_states[thrid[2]][idx] )#swap
        end
     end
    
end



In [6]:
 runnerdir = "/home/ghun245/RuNNer-testing/Brass_potential/"
 atomtype="Cu"
# pot = AbstractMLPotential(runnerdir,atomtype)
# RuNNer.getRuNNerenergy(runnerdir,1;input_idx=2)

"Cu"

In [18]:
pot_vector = []

for i =1:Threads.nthreads()
    potential=ParallelMLPotential(runnerdir,"Cu",i)
    push!(pot_vector,potential)
end


In [19]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot_vector[1] ; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]
    

results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)

 -------------------------------------------------------------
 ---------------------- Welcome to the -----------------------
     RuNNer Neural Network Energy Representation - RuNNer     
 ----------  (c) 2008-2022 Prof. Dr. Joerg Behler   ----------
 ----------  Georg-August-Universitaet Goettingen   ----------
 ----------           Theoretische Chemie           ----------
 ----------              Tammannstr. 6              ----------
 ----------        37077 Goettingen, Germany        ----------
 -------------------------------------------------------------
 -------------------------------------------------------------
  This program is free software: you can redistribute it and/or modify it
  under the terms of the GNU General Public License as published by the 
  Free Software Foundation, either version 3 of the License, or 
  (at your option) any later version.
    
  This program is distributed in the hope that it will be useful, but 
  WITHOUT ANY WARRANTY; without even the imp

   15 Cu  3  Cu Zn   0.000100  -1.000000   2.000000  11.338000
   16 Cu  3  Zn Zn   0.000100  -1.000000   2.000000  11.338000
   17 Cu  3  Cu Cu   0.003000  -1.000000   1.000000  11.338000
   18 Cu  3  Cu Zn   0.003000  -1.000000   1.000000  11.338000
   19 Cu  3  Zn Zn   0.003000  -1.000000   1.000000  11.338000
   20 Cu  3  Cu Cu   0.003000  -1.000000   2.000000  11.338000
   21 Cu  3  Cu Zn   0.003000  -1.000000   2.000000  11.338000
   22 Cu  3  Zn Zn   0.003000  -1.000000   2.000000  11.338000
   23 Cu  3  Cu Cu   0.008000  -1.000000   1.000000  11.338000
   24 Cu  3  Cu Zn   0.008000  -1.000000   1.000000  11.338000
   25 Cu  3  Zn Zn   0.008000  -1.000000   1.000000  11.338000
   26 Cu  3  Cu Cu   0.008000  -1.000000   2.000000  11.338000
   27 Cu  3  Cu Zn   0.008000  -1.000000   2.000000  11.338000
   28 Cu  3  Zn Zn   0.008000  -1.000000   2.000000  11.338000
   29 Cu  3  Cu Cu   0.008000   1.000000   2.000000  11.338000
   30 Cu  3  Cu Zn   0.008000   1.000000   2.000000  11

 -------------------------------------------------------------
 ---------------------- Welcome to the -----------------------
     RuNNer Neural Network Energy Representation - RuNNer     
 ----------  (c) 2008-2022 Prof. Dr. Joerg Behler   ----------
 ----------  Georg-August-Universitaet Goettingen   ----------
 ----------           Theoretische Chemie           ----------
 ----------              Tammannstr. 6              ----------
 ----------        37077 Goettingen, Germany        ----------
 -------------------------------------------------------------
 -------------------------------------------------------------
  This program is free software: you can redistribute it and/or modify it
  under the terms of the GNU General Public License as published by the 
  Free Software Foundation, either version 3 of the License, or 
  (at your option) any later version.
    
  This program is distributed in the hope that it will be useful, but 
  WITHOUT ANY WARRANTY; without even the imp

   14 Cu  3  Cu Cu   0.000100  -1.000000   2.000000  11.338000
   15 Cu  3  Cu Zn   0.000100  -1.000000   2.000000  11.338000
   16 Cu  3  Zn Zn   0.000100  -1.000000   2.000000  11.338000
   17 Cu  3  Cu Cu   0.003000  -1.000000   1.000000  11.338000
   18 Cu  3  Cu Zn   0.003000  -1.000000   1.000000  11.338000
   19 Cu  3  Zn Zn   0.003000  -1.000000   1.000000  11.338000
   20 Cu  3  Cu Cu   0.003000  -1.000000   2.000000  11.338000
   21 Cu  3  Cu Zn   0.003000  -1.000000   2.000000  11.338000
   22 Cu  3  Zn Zn   0.003000  -1.000000   2.000000  11.338000
   23 Cu  3  Cu Cu   0.008000  -1.000000   1.000000  11.338000
   24 Cu  3  Cu Zn   0.008000  -1.000000   1.000000  11.338000
   25 Cu  3  Zn Zn   0.008000  -1.000000   1.000000  11.338000
   26 Cu  3  Cu Cu   0.008000  -1.000000   2.000000  11.338000
   27 Cu  3  Cu Zn   0.008000  -1.000000   2.000000  11.338000
   28 Cu  3  Zn Zn   0.008000  -1.000000   2.000000  11.338000
   29 Cu  3  Cu Cu   0.008000   1.000000   2.000000  11

 Prediction Mode for all configurations
 -------------------------------------------------------------
 NNconfiguration:         1    Number of atoms:        13
 -------------------------------------------------------------
 begin
 atom      -0.00000005     -0.00000005     -0.00000003 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -0.00000752     -0.00000001      6.73662421 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom       1.86195381     -5.73051487      3.01271201 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -4.87467046     -3.54165303      3.01270453 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom      -4.87467052      3.54165288      3.01270450 Cu       0.00000000      0.00000000      0.00000000      0.00000000      0.00000000
 atom       1.86195371      5.73051485      3.01271195 Cu       0.00000000

...

   47 Zn  3  Cu Cu   0.025000   1.000000   1.000000  11.338000
   48 Zn  3  Cu Zn   0.025000   1.000000   1.000000  11.338000
   49 Zn  3  Zn Zn   0.025000   1.000000   1.000000  11.338000
   50 Zn  3  Cu Cu   0.025000   1.000000   2.000000  11.338000
   51 Zn  3  Cu Zn   0.025000   1.000000   2.000000  11.338000
   52 Zn  3  Zn Zn   0.025000   1.000000   2.000000  11.338000
   53 Zn  3  Cu Cu   0.025000  -1.000000   4.000000  11.338000
   54 Zn  3  Cu Zn   0.025000  -1.000000   4.000000  11.338000
   55 Zn  3  Zn Zn   0.025000  -1.000000   4.000000  11.338000
   56 Zn  3  Cu Cu   0.025000  -1.000000  16.000000  11.338000
   57 Zn  3  Cu Zn   0.025000  -1.000000  16.000000  11.338000
   58 Zn  3  Zn Zn   0.025000  -1.000000  16.000000  11.338000
   59 Zn  3  Cu Cu   0.025000   1.000000  16.000000  11.338000
   60 Zn  3  Cu Zn   0.025000   1.000000  16.000000  11.338000
   61 Zn  3  Zn Zn   0.025000   1.000000  16.000000  11.338000
   62 Zn  3  Cu Cu   0.045000   1.000000   1.000000  11

  85     0.00000000    0.00080888    0.00000870    0.00080888
  86     0.00000000    0.03955735    0.00828455    0.03955735
  87     0.00000000    0.02609251    0.00190869    0.02609251
  88     0.00000000    0.01370620    0.00024830    0.01370620
 Short range symmetry function values for element Zn
 Training set:  min           max       average         range 
   1     0.00000000   13.74941909    6.10144559   13.74941909
   2     0.00000000   13.26138613    2.83172747   13.26138613
   3     0.00000000    6.90332211    3.00984760    6.90332211
   4     0.00000000    6.41184911    1.33211751    6.41184911
   5     0.00000000    4.47980686    1.87800389    4.47980686
   6     0.00000000    3.96732506    0.80086245    3.96732506
   7     0.00000000    1.02528334    0.34139360    1.02528334
   8     0.00000000    0.73626447    0.12919931    0.73626447
   9     0.00000000    0.00687440    0.00035124    0.00687440
  10     0.00000000    0.00446719    0.00010946    0.00446719
  11     0.00000

 General job information:
 -------------------------------------------------------------
 Executing host    : SC421571            
 User name         : ghun245             
 Starting date     : 27. 9.2022
 Starting time     : 13 h  4 min 
 Working directory : /home/ghun245/RuNNer-testing/Brass_potential                
 -------------------------------------------------------------
 -------------------------------------------------------------
 Serial run requested
 -------------------------------------------------------------
 Reading control parameters from input.nn
 -------------------------------------------------------------
 General input parameters:
 -------------------------------------------------------------
 NNP generation                                           2
 Short range NN is on
 Electrostatic NN is off
 vdW corrections switched off
 -------------------------------------------------------------
 RuNNer nn_type_short                                     1
 RuNNer is st

Excessive output truncated after 531267 bytes.

Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [20]:
E=RuNNer.getenergy(pot_vector[1].dir, start_config,pot_vector[1].atomtype,pot_vector[1].index)

-0.51488321

In [21]:
parallel_states,a,v,r,delta_en,n_threads = parallel_equilibration(mc_states,move_strat,mc_params,pot_vector[1],ensemble,results)

(Any[MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(50.0, 6315.499705261945, Config{13, SphericalBC{Float64}, Float64}(StaticArraysCore.SVector{3, Float64}[[0.017341842803045113, -0.05096086637790216, 0.08647259726993559], [-0.012022061817491908, -0.09843291878302945, 6.996150797036031], [2.0397356145376726, -6.063368400253866, 3.0768847734050246], [-5.107634659246941, -3.579439452728729, 3.081994644713683], [-5.012002047442987, 3.6367785394635064, 3.1948230355964773], [1.8782861900827856, 5.816300536370538, 3.0386724221097774], [6.201756092154903, 0.1211242287921967, 3.0855364108590098], [-2.168981359258592, -5.867008857247427, -3.331364350433792], [-6.361473861937815, -0.09635891892275883, -3.1032648786674764], [-1.9464192298165877, 5.919097463079138, -3.2846096183514324], [4.89904438396108, 3.718969616134437, -3.1575839153847394], [5.2004585188946955, -3.706994497857551, -2.9181998105835416], [0.05456113331885808, 0.13474096981286354, -7.003533

In [22]:
function testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    Threads.@threads for i=1:4
        for j=1:100
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

function testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vec)
    for i=1:4
        for j=1:100
            mc_cycle!(parallel_states[i], move_strat, mc_params, pot_vec[i], ensemble, 1, a, v, r)
        end
    end
end

testrunner (generic function with 1 method)

In [23]:
@benchmark testrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 19.974 s (0.17% GC) to evaluate,
 with a memory estimate of 246.73 MiB, over 4002205 allocations.

In [24]:
@benchmark testparallelrunner(parallel_states,a,v,r,delta_en,n_threads,pot_vector)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 11.426 s (0.33% GC) to evaluate,
 with a memory estimate of 238.84 MiB, over 2968957 allocations.

In [25]:
function testrun(input_idx)
    run(`./RuNNer$input_idx.x $input_idx`)
end

testrun (generic function with 1 method)

In [26]:
#Threads.@threads
for j=1:200 
    Threads.@threads for i=1:8
        testrun(i)    
    end
end


In [16]:
# for i =1:8
#     cp("$(runnerdir)RuNNer.x","$(runnerdir)RuNNer$i.x")
# end